# **<font color='black'>COLOMBIA</font>**

In [1]:
import logging
import warnings

logging.getLogger("py4j").setLevel(logging.ERROR)
logging.getLogger("pyspark").setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

from datetime import date, datetime, timedelta, date
import pandas as pd
from pandas import Series
import numpy as np
from math import sqrt
import scipy.stats as sps
import seaborn as sns
import matplotlib as m
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.pyplot import figure
from matplotlib.pylab import rcParams
from datetime import datetime
from dateutil.relativedelta import relativedelta

import requests
from statsforecast.arima import AutoARIMA
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
import sklearn
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Paquetes (checar o que preciso e o que não preciso)
import pyspark.sql.functions as F
# from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


#logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)
import shutil

import os
import dotenv
from gcpspark import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date, date_sub, col

Dotenv installed
Loaded variables .env True
Pyspark installed
JAVA:  /usr/lib/jvm/temurin-8-jdk-amd64
DATAPROC:  True
Current GCP Project Name: cencosudx
Current ENVIRONMENT: staging


In [2]:
ENV = os.environ["ENVIRONMENT"]
BUCKET=f"{ENV}-bucket-dataproc-bigquery/dataproc/co_transformation"
# TABLE="data_analytics_default.BQ_CO_SM_SALES_02"
dataset_id = 'data_analytics_default'
table_id = 'BQ_CO_SM_SALES_01'
table_id2 = 'BQ_CO_SM_SALES_02'
TABLE=f"{dataset_id}.{table_id}"
TABLE2=f"{dataset_id}.{table_id2}"

In [3]:
# Spark Session
spark = create_pyspark(name="co_umv_transformation", connection="GCP")

File already exists at: /jars/spark-bigquery-with-dependencies_2.12-0.26.0.jar
File already exists at: /jars/gcs-connector-hadoop3-2.2.19.jar
Process: co_umv_transformation_1716241077788


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Data collection
df = (
    spark.read.format("bigquery")
    .option("table", TABLE)
    .load()
)
# print(df.count())
df = df.drop("_PARTITIONTIME", "_PARTITIONDATE")
df.printSchema()

# Data collection
df_historical = (
    spark.read.format("bigquery")
    .option("table", TABLE2)
    .load()
)
# print(df.count())
df_historical = df_historical.drop("_PARTITIONTIME", "_PARTITIONDATE")
df_historical.printSchema()

root
 |-- canal_venta: string (nullable = true)
 |-- tienda_venta: string (nullable = true)
 |-- sku_venta: string (nullable = true)
 |-- ean_venta: string (nullable = true)
 |-- fecha_venta: date (nullable = true)
 |-- hora_venta: string (nullable = true)
 |-- transaccion_venta: string (nullable = true)
 |-- pos_venta: string (nullable = true)
 |-- unidades_venta: double (nullable = true)
 |-- unidades_umb_venta: string (nullable = true)
 |-- monto_neto_venta: double (nullable = true)
 |-- monto_costo_venta: double (nullable = true)
 |-- id_promocion: string (nullable = true)
 |-- monto_promocion: double (nullable = true)

root
 |-- EAN: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- LOCAL_ID: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- UNITS: double (nullable = true)



In [5]:
# duplicates = df.groupBy(df.columns).count().filter("count > 1")
# num_duplicates = duplicates.count()

# if num_duplicates > 0:
#     print(f"The dataset contains {num_duplicates} duplicate rows.")
#     duplicates.show()
# else:
#     print("The dataset does not contain any duplicate rows.")

In [6]:
df = df.select("ean_venta", "sku_venta", "tienda_venta", "fecha_venta", "unidades_venta")
df = df.withColumnRenamed("tienda_venta","LOCAL_ID")\
        .withColumnRenamed("sku_venta","ITEM_ID")\
        .withColumnRenamed("ean_venta","EAN")\
        .withColumnRenamed("fecha_venta","DATE")\
        .withColumnRenamed("unidades_venta","UNITS")\
        .filter(F.col("UNITS")>0)

In [7]:
df.printSchema()

root
 |-- EAN: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- LOCAL_ID: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- UNITS: double (nullable = true)



In [21]:
df.write.format("bigquery") \
    .option("temporaryGcsBucket",BUCKET) \
    .option("table", TABLE2) \
    .mode("overwrite") \
    .save()

In [22]:
# total_sales = df.select(F.sum("UNITS")).collect()[0][0]
# average_sales = df.select(F.avg("UNITS")).collect()[0][0]
# min_sales = df.select(F.min("UNITS")).collect()[0][0]
# max_sales = df.select(F.max("UNITS")).collect()[0][0]
# std_dev_sales = df.select(F.stddev("UNITS")).collect()[0][0]
# total_records = df.count()

In [23]:
spark.stop()